In [ ]:
import os
import skimage.io
import matplotlib.pyplot as plt
import numpy as np

# Import Mask RCNN
from mrcnn.config import Config
import mrcnn.model as modellib
from mrcnn import utils
from mrcnn import visualize

%matplotlib inline

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print('Project Directory: {}'.format(ROOT_DIR))

# Root directory of the dataset
DATA_DIR = os.path.join(ROOT_DIR, "dataset/wad")
print('Data Directory: {}'.format(DATA_DIR))

# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")
print('Logs and Model Directory: {}'.format(LOGS_DIR))

## Configuration && Dataset

In [ ]:
from .context.project import wad_data


# WAD Configuration
class WADInferenceConfig(wad_data.WADConfig):
    BATCH_SIZE = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    
    
wad_config = WADInferenceConfig()

# Dataset
dataset = wad_data.WADDataset()
dataset.load_data_from_file(os.path.join(ROOT_DIR, 'project/last_run_validation.pkl'))
dataset.root_dir = os.path.join(DATA_DIR, 'train')
dataset.prepare()

print('Number of Images in Dataset: {}'.format(dataset.num_images))

## Load Model

In [ ]:
MODEL_WEIGHTS_PATH = os.path.join(LOGS_DIR, 'wad20180621T1404/mask_rcnn_wad_0001.h5')

# Create model in inference mode
model = modellib.MaskRCNN(mode="inference", config=wad_config, model_dir=LOGS_DIR)

model.load_weights(MODEL_WEIGHTS_PATH, by_name=True)

## Inference

In [ ]:
# Load a random image from the dataset
image_id = np.random.randint(0, dataset.num_images)
image = dataset.load_image(image_id)
gt_masks = dataset.load_mask(image_id)

print('Running detection on image {} (filename: {})'.format(image_id, dataset.image_info[image_id]['path']))

results = model.detect([image], verbose=1)[0]

visualize.display_instances(image, results['rois'], results['masks'], results['class_ids'], 
                            wad_data.index_to_classes, results['scores'])

In [ ]:
# Calculate Average Precision (AP) for each image
APs = []
for image_id in range(dataset.num_images):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, wad_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, wad_config), 0)
    
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
    print('Image: {0:3d}\tAP: {1:1.4f}'.format(image_id, AP))
    
# Calculate mAP score for dataset
print("mAP: ", np.mean(APs))